In [ ]:
import keras
import numpy as np
from os.path import join
import tensorflow as tf
import random
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay, classification_report

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xvzf cifar-10-python.tar.gz
!rm cifar-10-python.tar.gz

--2024-09-19 01:55:40--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  49.0MB/s    in 3.5s    

2024-09-19 01:55:44 (46.8 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [ ]:
base_dir = "cifar-10-batches-py"

In [ ]:
data_train = []
labels_train = []
for i in range(1,6):
    data_dir = "data_batch_"+str(i)
    data_dict = unpickle(join(base_dir,data_dir))
    data_train.append(data_dict[b'data'])
    labels_train.append(data_dict[b'labels'])

data_train = np.concatenate(data_train)
labels_train = np.concatenate(labels_train)

In [ ]:
data_test_dic = unpickle(join(base_dir,"test_batch"))
data_test = data_test_dic[b'data']
labels_test = data_test_dic[b'labels']

data_test = np.array(data_test)
labels_test = np.array(labels_test)

In [ ]:
data_train = data_train.reshape((50000, 3, 32, 32))
data_test = data_test.reshape((10000, 3, 32, 32))

data_train = data_train.transpose((0, 2, 3, 1))
data_test = data_test.transpose((0, 2, 3, 1))

In [ ]:
labels_train =  keras.utils.to_categorical(labels_train)
labels_test =  keras.utils.to_categorical(labels_test)

In [ ]:
inputs = keras.Input(shape=(32, 32, 3))

vgg16 = keras.applications.VGG19(include_top=False, weights='imagenet', input_tensor=inputs)

In [ ]:
for layer in vgg16.layers:
   layer.trainable = False

In [ ]:
model_adam = keras.Sequential([
    vgg16,
    keras.layers.Flatten(),
      keras.layers.Dense(10, activation='softmax')
])

In [ ]:
epochs=50
learn_rate=0.001

adam=keras.optimizers.AdamW(learning_rate=learn_rate)
model_adam.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:

history_model_adam = model_adam.fit(data_train, labels_train, epochs=50, batch_size=128, validation_split=0.2)


In [ ]:
# Evaluate the model_cnn
test_loss, test_acc = model_adam.evaluate(data_test, labels_test)
print('Test accuracy:', test_acc)

In [ ]:
plt.figure(figsize=(20, 5))

# Primeiro gráfico (Loss)
plt.subplot(1, 2, 1)
plt.plot(history_model_adam.history['loss'])
plt.plot(history_model_adam.history['val_loss'])
plt.title('Loss do Modelo', color='white')  # Cor do título
plt.ylabel('Loss', color='white')           # Cor do label do eixo Y
plt.xlabel('Épocas', color='white')         # Cor do label do eixo X
plt.legend(['Train', 'Val'], loc='upper left')

# Mudar cor dos eixos para branco
ax = plt.gca()
ax.set_facecolor('black')
ax.spines['bottom'].set_color('white')
ax.spines['top'].set_color('white')
ax.spines['right'].set_color('white')
ax.spines['left'].set_color('white')
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')

# Segundo gráfico (Accuracy)
plt.subplot(1, 2, 2)
plt.plot(history_model_adam.history['accuracy'])
plt.plot(history_model_adam.history['val_accuracy'])
plt.title('Acurácia do Modelo', color='white')
plt.ylabel('Acurácia', color='white')
plt.xlabel('Épocas', color='white')
plt.legend(['Train', 'Val'], loc='upper left')

# Mudar cor dos eixos para branco
ax = plt.gca()
ax.set_facecolor('black')
ax.spines['bottom'].set_color('white')
ax.spines['top'].set_color('white')
ax.spines['right'].set_color('white')
ax.spines['left'].set_color('white')
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')

# Alterar a cor do fundo da figura principal
plt.gcf().set_facecolor('black')

plt.show()
